In [9]:
import tkinter as tk
from tkinter import filedialog, messagebox
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import random
import os

# Age group settings
AGE_GROUPS = {
    "Age 6-7": {"ops": ["+", "-"], "max": 10},
    "Age 8-9": {"ops": ["+", "-", "*"], "max": 20},
    "Age 9-10": {"ops": ["+", "-", "*"], "max": 30},
    "Age 10-11": {"ops": ["+", "-", "*", "/"], "max": 50},
}

# Generate questions & answers
def generate_questions(num_questions, operations, max_number):
    questions = []
    answers = []
    for i in range(1, num_questions + 1):
        a = random.randint(1, max_number)
        b = random.randint(1, max_number)
        op = random.choice(operations)

        if op == "/":
            b = random.randint(1, max(2, max_number // 2))
            a = b * random.randint(1, max_number // b)

        expr = f"{a} {op} {b}"
        display_op = {"*": "×", "/": "÷"}.get(op, op)
        display_expr = f"{a} {display_op} {b}"
        question = f"{i}. {display_expr} = ___"
        answer = f"{i}. {int(eval(expr)) if op == '/' else eval(expr)}"
        questions.append(question)
        answers.append(answer)
    return questions, answers

# Save PDF with headers for each worksheet in answer key
def save_pdf(filename, all_worksheets, all_answers, title):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter

    for idx, (questions, _) in enumerate(all_worksheets):
        c.setFont("Helvetica-Bold", 14)
        c.drawString(72, height - 50, f"{title} Worksheet #{idx+1}")
        c.setFont("Helvetica", 12)
        c.drawString(72, height - 70, "Name: ______________________________             Date: _______________________________")

        x1, x2 = 72, 300
        y = height - 110
        col1 = questions[:15]
        col2 = questions[15:]

        for i in range(15):
            if i < len(col1):
                num, q = col1[i].split(". ", 1)
                c.setFont("Helvetica-Bold", 12)
                c.drawString(x1, y, f"{num}.")
                c.setFont("Helvetica", 12)
                c.drawString(x1 + 20, y, q)
            if i < len(col2):
                num2, q2 = col2[i].split(". ", 1)
                c.setFont("Helvetica-Bold", 12)
                c.drawString(x2, y, f"{num2}.")
                c.setFont("Helvetica", 12)
                c.drawString(x2 + 20, y, q2)
            y -= 40
        c.showPage()

    # Answer Key Section with worksheet headers
    for idx, answers in enumerate(all_answers):
        c.setFont("Helvetica-Bold", 14)
        c.drawString(72, height - 50, f"📝 Answer Key - Worksheet #{idx + 1}")
        c.setFont("Helvetica", 12)

        y = height - 80
        col_width = 180
        col_count = 3

        for i, ans in enumerate(answers):
            col = i % col_count
            row = i // col_count
            x = 72 + col * col_width
            y_line = height - 80 - row * 20

            if y_line < 72:
                c.showPage()
                y = height - 80
                row = 0
                y_line = height - 80

            num, val = ans.split(". ", 1)
            c.setFont("Helvetica-Bold", 12)
            c.drawString(x, y_line, f"{num}.")
            c.setFont("Helvetica", 12)
            c.drawString(x + 20, y_line, val)

        c.showPage()  # new page for each worksheet's answers

    c.save()


# Generate all worksheets by age group
def generate_all_worksheets():
    try:
        output_folder = filedialog.askdirectory(title="Select folder to save PDFs")
        if not output_folder:
            return

        for age_group, config in AGE_GROUPS.items():
            count = int(entries[age_group].get())
            all_worksheets = []
            all_answers = []

            for _ in range(count):
                qs, ans = generate_questions(30, config["ops"], config["max"])
                all_worksheets.append((qs, ans))
                all_answers.append(ans)

            filename = os.path.join(output_folder, f"Math_{age_group.replace(' ', '_')}.pdf")
            save_pdf(filename, all_worksheets, all_answers, f"Math Worksheet - {age_group}")

        messagebox.showinfo("Done", "All worksheets generated and saved.")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# GUI Setup
root = tk.Tk()
root.title("Math Worksheet Generator by Age Group")

entries = {}

tk.Label(root, text="Enter number of worksheets per age group:").grid(row=0, column=0, columnspan=2, pady=10)

for idx, age_group in enumerate(AGE_GROUPS.keys(), start=1):
    tk.Label(root, text=age_group + ":").grid(row=idx, column=0, sticky="e")
    entry = tk.Entry(root, width=5)
    entry.insert(0, "1")
    entry.grid(row=idx, column=1)
    entries[age_group] = entry

tk.Button(root, text="Generate Worksheets", command=generate_all_worksheets).grid(row=idx+1, column=0, columnspan=2, pady=10)

root.mainloop()
